In [ ]:
# !source $linkedin_VENV

In [1]:
!mkdir documents/
!cd documents/

## 1. Installing and importing required modules

In [2]:
!pip3 install -qq linkedin_scraper
!pip3 install -qq selenium
!apt install -yq chromium-chromedriver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb
  udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 9 newly installed, 0 to remove and 34 not upgraded.
Need to get 26.4 MB of archives.
After this operation, 116 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/ma

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from linkedin_scraper import actions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import os
import sys
import json

## 2. Initial setup for the project

In [4]:
# Setting the path of the chromium webdriver in the system

path = "/lib/chromium-browser/chromedriver"
sys.path.insert(0,path)

In [5]:
# Configuring the options for the chromium webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [6]:
# Starting the webdriver

driver = webdriver.Chrome(options = chrome_options)

In [7]:
cookie_val = input("Please input the cookie: ")

In [8]:
# Logging into linkedin

driver.get("https://in.linkedin.com/?src=go-pa&trk=sem-ga_campid.14650114788_asid.151761418307_crid.657403558721_kw.linkedin%20login_d.c_tid.kwd-12704335873_n.g_mt.e_geo.9182462&mcid=6844056167778418689&cid=&gad_source=1&gclid=EAIaIQobChMI6I7N8uPLhAMVh6lmAh34lw7MEAAYASAAEgL2NvD_BwE&gclsrc=aw.ds")

new_cookie = {
    "name" : "li_at",
    "value" : cookie_val,
    "domain" : ".linkedin.com",
    "path" : "/",
    "secure" : True
}

driver.add_cookie(new_cookie)

driver.refresh()

In [9]:
# driver.current_url

'https://in.linkedin.com/feed/?trk=sem-ga_campid.14650114788_asid.151761418307_crid.657403558721_kw.linkedin%20login_d.c_tid.kwd-12704335873_n.g_mt.e_geo.9182462'

In [10]:
# Example profiles to be scraped

manoj_url = "https://www.linkedin.com/in/mkpentapalli/"
thiru_url = "https://www.linkedin.com/in/thirumalaraomodepalli/"
jennifer_url = "https://www.linkedin.com/in/jjcmo/"
satya_url = "https://www.linkedin.com/in/satyanadella/"
bill_url = "https://www.linkedin.com/in/williamhgates/"

In [ ]:
print(driver.current_url)

https://www.linkedin.com/checkpoint/challenge/AgFWzTkluvr8twAAAY3qw0mstMN4ugB1F0iE0EoAPMZz1FyVqud3Cpww3IWercEVYfCVSav9m__AHrBXmTCjRXy5cTNT5w?ut=1V55KPM5qHEr81


## 3. Required functions

### 3.1 Functions for generic elements

In [11]:
# Type of element taken:

element_type_dict = {
    0 : "CSS_SELECTOR",
    1 : "CLASS_NAME",
    2 : "ID",
    3 : "TAG_NAME",
    4 : "XPATH"
}

In [12]:
# getting the child element from the parent element

def get_element(parent_element=None, child_element_name="", element_type=1, wait_time=8):
    try:
        by_attr = getattr(By, element_type_dict[element_type])
        child_element = WebDriverWait(parent_element, wait_time).until(EC.presence_of_element_located((by_attr, child_element_name)))
        return child_element
    except Exception as e:
        print(f"{type(e).__name__} occurred for the child element name {child_element_name}.")
        pass

In [13]:
# getting the child elements from the parent element

def get_elements(parent_element=None, child_elements_name="", element_type=1, wait_time=8):
    try:
        by_attr = getattr(By, element_type_dict[element_type])
        child_elements = WebDriverWait(parent_element, wait_time).until(EC.presence_of_all_elements_located((by_attr, child_elements_name)))
        return child_elements
    except Exception as e:
        print(f"{type(e).__name__} occurred for the child element name {child_elements_name}.")
        pass

In [14]:
# getting elements from a list of parent elements

def get_elements_from_list(parent_list = [], child_element = "", element_type = 1, wait_time = 3):
    element_list = []
    for element in parent_list:
        element_list_temp = get_elements(element, child_element, element_type)
        content = ""
        for temp_element in element_list_temp:
            content += temp_element.text + "\n"
        element_list.append(content)

    return element_list

### 3.2 Functions for specific elements:

In [15]:
# Name

def get_name(parent_element = None) -> str:
    name = None
    if parent_element:
        name_elem = get_element(parent_element, "h1", 3)
        name = name_elem.text if name_elem else None
    return name

In [16]:
# Location

def get_location(parent_element = None) -> str:
    location = None
    if parent_element:
        location_elem = get_element(parent_element, ".text-body-small.inline.t-black--light.break-words", 0)
        location = location_elem.text if location_elem else None
    return location

In [17]:
# Intro

def get_intro(parent_element = None) -> str:
    intro = None
    if parent_element:
        intro_elem = get_element(parent_element, ".text-body-medium.break-words", 0)
        intro = intro_elem.text if intro_elem else None
    return intro

In [18]:
# Work Preference

def get_work_preference(parent_element = None) -> str:
    photo_element = get_element(parent_element, "pv-top-card--photo", 1)
    img_element = get_element(photo_element, "img", 3)
    work_preference = img_element.get_attribute("alt") if img_element else ""
    work_preference_text = None
    if("#OPEN_TO_WORK" in work_preference):
        work_preference_text = "Open to work"
    elif("#HIRING" in work_preference):
        work_preference_text = "Hiring"
    return work_preference_text

In [19]:
# Contact details

def get_contact_details(parent_element = None):
    contact_details = None
    contact_card = get_element(parent_element, ".pv-profile-section__section-info.section-info",0)
    urls_text_list = get_elements(contact_card,"h3",0)
    urls_text = []
    for text in urls_text_list:
        urls_text.append(text.text)
    urls_list = get_elements(contact_card,"a",0)
    urls = []
    for url in urls_list:
        urls.append(url.get_attribute("href"))
    contact_details = dict(zip(urls_text,urls))
    return contact_details

In [20]:
# About section

def get_about(parent_element = None) -> str:
    about = None
    if parent_element:
        about_element = get_element(parent_element, ".display-flex.ph5.pv3", 0)
        about_element_span = get_element(about_element, "span", 3)
        about = about_element_span.text if about_element_span else ""
    return about

### 3.3 Caller function

In [21]:
def getPersonalDetails(driver = None, profile_url = "") -> str:
    name = None
    location = None
    intro = None
    work_preference = None
    contact = None
    about = None
    experiences = None
    education = None
    personal_details = dict()
    details = ""


    driver.get(profile_url)

    # Scraping for profile details:

    top_card = get_element(driver, "scaffold-layout__main")
    top_panel = get_element(top_card, ".mt2.relative", 0)

    # Name:
    name = get_name(top_panel)
    personal_details["Name"] = name

    # Location:
    location = get_location(top_panel)
    personal_details["Location"] = location

    # Intro:
    intro = get_intro(top_panel)
    personal_details["Headline"] = intro

    # Work Preference:
    work_preference = get_work_preference(top_card)
    personal_details["Work_Preference"] = work_preference

    # Contact:
    profile_contact_url = profile_url + "/overlay/contact-info/"
    driver.get(profile_contact_url)
    contact = get_contact_details(driver)
    if contact:
        personal_details["Contact_Details"] = contact

    # About:
    driver.get(profile_url)
    top_card = get_element(driver, "scaffold-layout__main")
    about = get_about(top_card)
    if about:
        personal_details["About"] = about

    # Experiences:
    experiences_url = profile_url + "details/experience/"
    driver.get(experiences_url)
    experiences_top_card = get_element(driver, "scaffold-layout__main",1)
    experiences_list_card = get_elements(experiences_top_card, ".pvs-list__paged-list-item.artdeco-list__item.pvs-list__item--line-separated.pvs-list__item--one-column", 0)
    experiences = get_elements_from_list(experiences_list_card,"visually-hidden")
    if experiences:
        personal_details["Experience"] = ""
        for item in experiences:
            personal_details["Experience"] += item

    # Education:
    education_url = profile_url + "details/education/"
    driver.get(education_url)
    education_top_card = get_element(driver, "scaffold-layout__main")
    education_list_card = get_elements(education_top_card,".pvs-list__paged-list-item.artdeco-list__item.pvs-list__item--line-separated.pvs-list__item--one-column",0)
    education = get_elements_from_list(education_list_card,"visually-hidden",1)
    if education:
        personal_details["Education"] = ""
        for item in education:
            personal_details["Education"] += item

    for key in personal_details.keys():
        # print(key, "\n\n\n", personal_details[key])
        details += f"{key}:\n\n{personal_details[key]}\n\n\n"

    return details

## 4. Creating text files

In [22]:
names = []
urls = []

# Example urls and names:
# "Kamil Imtiaz"
# "https://www.linkedin.com/in/kamilimtiaz/"
names = ["Gautam Kumar"]
urls = ["https://www.linkedin.com/in/gautam-kumar-16203213b/"]

# names = ["Manoj Kumar Pentapalli", "Thirumala Rao Modepalli", "Jennifer Johnson", "Satya Nadella", "Bill Gates"]
# urls = [manoj_url, thiru_url, jennifer_url, satya_url, bill_url]

In [ ]:
# # Take input for urls and names:

# n = int(input("Enter number of profiles to be given: "))

# print("Input for profile Names:\n")

# for i in range(n):
#     temp = input()
#     names.append(temp)

# print("Input for urls:\n")

# for i in range(n):
#     temp = input()
#     urls.append(temp)

In [23]:
for i in range(len(urls)):
    personal_details = getPersonalDetails(driver,urls[i])
    with open(f"documents/{names[i]}.txt","w") as f:
        f.write(f"{personal_details}")

In [ ]:
# driver.quit()